# Classification

Smile's classification algorithms are in the package `smile.classification` and all algorithms implement the interface `Classifier` that has the method `predict` to predict the class label of an instance. An overloaded version in `SoftClassifier` can also calculate the a posteriori probabilities besides the class label.

Some algorithms with online learning capability also implement the interface `OnlineClassifier`. Online learning is a model of induction that learns one instance at a time. The method `update` updates the model with a new instance.

The high-level operators are defined in Scala package object of `smile.classification`. In what follows, we will discuss each algorithm, their high-level Scala API, and examples.

In [ ]:
import $ivy.`com.github.haifengl::smile-scala:2.2.2`
import $ivy.`org.slf4j:slf4j-simple:1.7.30`  

import scala.language.postfixOps
import smile._
import smile.math._
import smile.math.distance._
import smile.math.kernel._
import smile.math.matrix._
import smile.math.matrix.Matrix._
import smile.math.rbf._
import smile.stat.distribution._
import smile.data._
import smile.data.formula._
import smile.data.measure._
import smile.data.`type`._
import smile.base.cart.SplitRule
import smile.base.mlp._
import smile.base.rbf.RBF
import smile.classification._
import smile.feature._
import smile.validation._

In below examples, we will use the `iris` data as samples.

In [ ]:
val iris = read.arff("data/weka/iris.arff")

We also define a `Formula` that specifies the class labels and predictors. In the example, the right-hand side of formula is empty, which means that all the rest of variables in the data frame will be used as predictors.

In [ ]:
val formula: Formula = "class" ~
val x = formula.x(iris).toArray
val y = formula.y(iris).toIntArray

Lastly, we extract the predictors and class labels with the formula.

## Nearest Neighbor

The k-nearest neighbor algorithm (k-NN) is a method for classifying objects by a majority vote of its neighbors, with the object being assigned to the class most common amongst its `k` nearest neighbors (`k` is typically small). k-NN is a type of instance-based learning, or lazy learning where the function is only approximated locally and all computation is deferred until classification.val mat = matrix(100, 100)

In [ ]:
cv.classification(10, x, y) { case (x, y) => knn(x, y, 3) }

The funciton `cv.classification(k, x , y)` performs `k`-fold cross validation and takes a code block that trains the model.

## Decision Trees

A decision tree can be learned by splitting the training set into subsets based on an attribute value test. This process is repeated on each derived subset in a recursive manner called recursive partitioning. The recursion is completed when the subset at a node all has the same value of the target variable, or when splitting no longer adds value to the predictions.

The algorithms that are used for constructing decision trees usually work top-down by choosing a variable at each step that is the next best variable to use in splitting the set of items. "Best" is defined by how well the variable splits the set into homogeneous subsets that have the same value of the target variable. Different algorithms use different formulae for measuring "best". Used by the CART (Classification and Regression Tree) algorithm, Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it were randomly labeled according to the distribution of labels in the subset. Gini impurity can be computed by summing the probability of each item being chosen times the probability of a mistake in categorizing that item. It reaches its minimum (zero) when all cases in the node fall into a single target category. Information gain is another popular measure, used by the ID3, C4.5 and C5.0 algorithms. Information gain is based on the concept of entropy used in information theory. For categorical variables with different number of levels, however, information gain are biased in favor of those attributes with more levels. Instead, one may employ the information gain ratio, which solves the drawback of information gain.
```
      def cart(formula: Formula, data: DataFrame, splitRule: SplitRule = SplitRule.GINI, maxDepth: Int = 20, maxNodes: Int = 0, nodeSize: Int = 5): DecisionTree
``` 
where `maxDepth` controls the maximum depth of the tree as a way of regularization. Similarly, the parameter `maxNodes`, if positive, is the maximum number of leaf nodes in the tree as a regularization control. If 0 or negative, `maxNodes` will be ignored. The parameter `nodeSize` controls the minimum number of samples in the leaf nodes.

In [ ]:
cv.classification(10, formula, iris) { case (formula, data) => cart(formula, data) }

Decision tree techniques have a number of advantages over many of those alternative techniques.

- **Simple to understand and interpret**:
In most cases, the interpretation of results summarized in a tree is very simple. This simplicity is useful not only for purposes of rapid classification of new observations, but can also often yield a much simpler "model" for explaining why observations are classified or predicted in a particular manner.

- **Able to handle both numerical and categorical data**:
Other techniques are usually specialized in analyzing datasets that have only one type of variable.

- **Nonparametric and nonlinear**:
The final results of using tree methods for classification or regression can be summarized in a series of (usually few) logical if-then conditions (tree nodes). Therefore, there is no implicit assumption that the underlying relationships between the predictor variables and the dependent variable are linear, follow some specific non-linear link function, or that they are even monotonic in nature. Thus, tree methods are particularly well suited for data mining tasks, where there is often little a priori knowledge nor any coherent set of theories or predictions regarding which variables are related and how. In those types of data analytics, tree methods can often reveal simple relationships between just a few variables that could have easily gone unnoticed using other analytic techniques.

One major problem with classification and regression trees is their high variance. Often a small change in the data can result in a very different series of splits, making interpretation somewhat precarious. Besides, decision-tree learners can create over-complex trees that cause over-fitting. Mechanisms such as pruning are necessary to avoid this problem. Another limitation of trees is the lack of smoothness of the prediction surface.

Some techniques such as bagging, boosting, and random forest use more than one decision tree for their analysis.

## Random Forest

Random forest is an ensemble classifier that consists of many decision trees and outputs the majority vote of individual trees. The method combines bagging idea and the random selection of features.

Each tree is constructed using the following algorithm:

  - If the number of cases in the training set is `N`, randomly sample `N` cases with replacement from the original data. This sample will be the training set for growing the tree.
  - If there are `M` input variables, a number `m` << `M` is specified such that at each node, `m` variables are selected at random out of the `M` and the best split on these m is used to split the node. The value of `m` is held constant during the forest growing.
  - Each tree is grown to the largest extent possible. There is no pruning.
    
where ntrees is the number of trees, and mtry is the number of attributed randomly selected at each node to choose the best split. Although the original random forest algorithm trains each tree fully without pruning, it is useful to control the tree size some times, which can be achieved by the parameter maxNodes. The tree can also be regularized by limiting the minimum number of observations in trees' terminal nodes with the parameter nodeSize. When subsample = 1.0, we use the sampling with replacement to train each tree as described above. If subsample < 1.0, we instead select a subset of samples (without replacement) to train each tree. If the classes are not balanced, the user should provide the classWeight (proportional to the class priori) so that the sampling is done in stratified way. Otherwise, small classes may be not sampled sufficiently.

In [ ]:
cv.classification(10, formula, iris) { case (formula, data) => randomForest(formula, data) }

The advantages of random forest are:

- For many data sets, it produces a highly accurate classifier.
- It runs efficiently on large data sets.
- It can handle thousands of input variables without variable deletion.
- It gives estimates of what variables are important in the classification.
- It generates an internal unbiased estimate of the generalization error as the forest building progresses.
- It has an effective method for estimating missing data and maintains accuracy when a large proportion of the data are missing.

The disadvantages are

- Random forests are prone to over-fitting for some datasets. This is even more pronounced on noisy data.
- For data including categorical variables with different number of levels, random forests are biased in favor of those attributes with more levels. Therefore, the variable importance scores from random forest are not reliable for this type of data.

## Gradient Boosted Trees

The idea of gradient boosting originated in the observation that boosting can be interpreted as an optimization algorithm on a suitable cost function. In particular, the boosting algorithms can be abstracted as iterative functional gradient descent algorithms. That is, algorithms that optimize a cost function over function space by iteratively choosing a function (weak hypothesis) that points in the negative gradient direction Gradient boosting is typically used with CART regression trees of a fixed size as base learners.

Generic gradient boosting at the *t-th* step would fit a regression tree to pseudo-residuals. Let `J` be the number of its leaves. The tree partitions the input space into J disjoint regions and predicts a constant value in each region. The parameter `J` controls the maximum allowed level of interaction between variables in the model. With `J = 2` (decision stumps), no interaction between variables is allowed. With `J = 3` the model may include effects of the interaction between up to two variables, and so on. Hastie et al. comment that typically `4 ≤ J ≤ 8` work well for boosting and results are fairly insensitive to the choice of in this range, `J = 2` is insufficient for many applications, and `J > 10` is unlikely to be required.

Fitting the training set too closely can lead to degradation of the model's generalization ability. Several so-called regularization techniques reduce this over-fitting effect by constraining the fitting procedure. One natural regularization parameter is the number of gradient boosting iterations T (i.e. the number of trees in the model when the base learner is a decision tree). Increasing T reduces the error on training set, but setting it too high may lead to over-fitting. An optimal value of T is often selected by monitoring prediction error on a separate validation data set.

Another regularization approach is the shrinkage which times a parameter `η` (called the "learning rate") to update term. Empirically it has been found that using small learning rates (such as `η < 0.1`) yields dramatic improvements in model's generalization ability over gradient boosting without shrinking (`η = 1`). However, it comes at the price of increasing computational time both during training and prediction: lower learning rate requires more iterations.

In [ ]:
cv.classification(10, formula, iris) { case (formula, data) => gbm(formula, data) }

Soon after the introduction of gradient boosting Friedman proposed a minor modification to the algorithm, motivated by Breiman's bagging method. Specifically, he proposed that at each iteration of the algorithm, a base learner should be fit on a subsample of the training set drawn at random without replacement. Friedman observed a substantial improvement in gradient boosting's accuracy with this modification.

Subsample size is some constant fraction `f` of the size of the training set. When `f = 1`, the algorithm is deterministic and identical to the one described above. Smaller values of f introduce randomness into the algorithm and help prevent over-fitting, acting as a kind of regularization. The algorithm also becomes faster, because regression trees have to be fit to smaller datasets at each iteration. Typically, `f` is set to 0.5, meaning that one half of the training set is used to build each base learner.

Also, like in bagging, sub-sampling allows one to define an out-of-bag estimate of the prediction performance improvement by evaluating predictions on those observations which were not used in the building of the next base learner. Out-of-bag estimates help avoid the need for an independent validation dataset, but often underestimate actual performance improvement and the optimal number of iterations.

## SVM

The basic support vector machine (SVM) is a binary linear classifier which chooses the hyperplane that represents the largest separation, or margin, between the two classes. If such a hyperplane exists, it is known as the maximum-margin hyperplane and the linear classifier it defines is known as a maximum margin classifier.

In [ ]:
val train = read.libsvm("data/libsvm/svmguide1")
val test  = read.libsvm("data/libsvm/svmguide1.t")

val n = train.size
val x = Array.ofDim[Double](n, 4)
val y = Array.ofDim[Int](n)
(0 until n) foreach { i =>
    val sample = train.get(i)
    sample.x.forEach(e => x(i)(e.i) = e.x)
    y(i) = if (sample.label > 0) +1 else -1
}

val testn = test.size
val testx = Array.ofDim[Double](testn, 4)
val testy = Array.ofDim[Int](testn)
(0 until testn) foreach { i =>
    val sample = test.get(i)
    sample.x.forEach(e => testx(i)(e.i) = e.x)
    testy(i) = if (sample.label > 0) +1 else -1
}

val kernel = new GaussianKernel(90)
val model = SVM.fit(x, y, kernel, 100, 1E-3)

val prediction = Validation.test(model, testx)
println(s"accuracy = ${accuracy(testy, prediction)}")

If there exists no hyperplane that can perfectly split the positive and negative instances, the soft margin method will choose a hyperplane that splits the instances as cleanly as possible, while still maximizing the distance to the nearest cleanly split instances.

The nonlinear SVMs are created by applying the kernel trick to maximum-margin hyperplanes. The resulting algorithm is formally similar, except that every dot product is replaced by a nonlinear kernel function. This allows the algorithm to fit the maximum-margin hyperplane in a transformed feature space. The transformation may be nonlinear and the transformed space be high dimensional. For example, the feature space corresponding Gaussian kernel is a Hilbert space of infinite dimension. Thus though the classifier is a hyperplane in the high-dimensional feature space, it may be nonlinear in the original input space. Maximum margin classifiers are well regularized, so the infinite dimension does not spoil the results.

The effectiveness of SVM depends on the selection of kernel, the kernel's parameters, and soft margin parameter `C`. Given a kernel, best combination of C and kernel's parameters is often selected by a grid-search with cross validation.

## Logistic Regression

Logistic regression (logit model) is a generalized linear model used for binomial regression. Logistic regression applies maximum likelihood estimation after transforming the dependent into a logit variable. A logit is the natural log of the odds of the dependent equaling a certain value or not (usually 1 in binary logistic models, the highest value in multinomial models). In this way, logistic regression estimates the odds of a certain event (value) occurring.
```
    def logit(x: Array[Array[Double]], y: Array[Int], lambda: Double = 0.0, tol: Double = 1E-5, maxIter: Int = 500): LogisticRegression
```
where the parameter `lambda` (> 0) gives a "regularized" estimate of linear weights which often has superior generalization performance, especially when the dimensionality is high.

In [ ]:
cv.classification(10, x, y) { case (x, y) => logit(x, y) }

Logistic regression has many analogies to ordinary least squares (OLS) regression. Unlike OLS regression, however, logistic regression does not assume linearity of relationship between the raw values of the independent variables and the dependent, does not require normally distributed variables, does not assume homoscedasticity, and in general has less stringent requirements.

Compared with linear discriminant analysis, logistic regression has several advantages:

- It is more robust: the independent variables don't have to be normally distributed, or have equal variance in each group
- It does not assume a linear relationship between the independent variables and dependent variable.
- It may handle nonlinear effects since one can add explicit interaction and power terms.

However, it requires much more data to achieve stable, meaningful results.

Logistic regression also has strong connections with neural network and maximum entropy modeling. For example, binary logistic regression is equivalent to a one-layer, single-output neural network with a logistic activation function trained under log loss. Similarly, multinomial logistic regression is equivalent to a one-layer, softmax-output neural network.

Logistic regression estimation also obeys the maximum entropy principle, and thus logistic regression is sometimes called "maximum entropy modeling", and the resulting classifier the "maximum entropy classifier".

## Neural Networks

A multilayer perceptron neural network consists of several layers of nodes, interconnected through weighted acyclic arcs from each preceding layer to the following, without lateral or feedback connections. Each node calculates a transformed weighted linear combination of its inputs (output activations from the preceding layer), with one of the weights acting as a trainable bias connected to a constant input. The transformation, called activation function, is a bounded non-decreasing (non-linear) function, such as the sigmoid functions (ranges from `0` to `1`). Another popular activation function is hyperbolic tangent which is actually equivalent to the sigmoid function in shape but ranges from `-1` to `1`. More specialized activation functions include radial basis functions which are used in RBF networks.

For neural networks, the input patterns usually should be scaled/standardized. Commonly, each input variable is scaled into interval `[0, 1]` or to have mean `0` and standard deviation `1`.

In [ ]:
val pendigits = read.csv("data/classification/pendigits.txt", delimiter = '\t', header = false)

val formula: Formula = "V17" ~
val data = formula.x(pendigits).toArray
val y = formula.y(pendigits).toIntArray

val scaler = WinsorScaler.fit(data, 0.01, 0.99)
val x = scaler.transform(data)

val p = x(0).length
val k = MathEx.max(y) + 1

MathEx.setSeed(19650218); // to get repeatable results.
cv.classification(10, x, y) { (x, y) =>
  val model = new MLP(p,
    Layer.sigmoid(50),
    Layer.mle(k, OutputFunction.SIGMOID)
  )

  (0 until 8) foreach { eopch =>
     val permutation = MathEx.permutate(x.length)
     for (i <- permutation) {
       model.update(x(i), y(i))
     }
  }

  model
}